In [38]:
# -*- coding: UTF-8 -*-
"""Evaluation performance of the most popular classification algorithms with
    artificial dataset generated by sklearn.datasets.make_classification()"""
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RepeatedStratifiedKFold, cross_validate
from sklearn import metrics
from sklearn.datasets import make_classification
from sklearn.ensemble import GradientBoostingClassifier, HistGradientBoostingClassifier, RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

In [39]:
# Define test classification dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=5, n_redundant=2, random_state=12)
print(f"Summarize the dataset: X.shape is {X.shape}; y.shape is {y.shape}")
# Split for train/test data as 80% and 20 %
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=12)

Summarize the dataset: X.shape is (1000, 10); y.shape is (1000,)


In [40]:
# Instances of the classification models
lr = LogisticRegression()
rf = RandomForestClassifier()
gb = GradientBoostingClassifier()
hist_gb = HistGradientBoostingClassifier()
xgb = XGBClassifier()
lgb = LGBMClassifier()
catgb = CatBoostClassifier(verbose=0, n_estimators=200)
mlp = MLPClassifier(max_iter=1500, solver='adam')

models = (lr, rf, gb, hist_gb, xgb, lgb, catgb, mlp)
models_tuple = ('LogisticRegression',
                'RandomForestClassifier',
                'GradientBoostingClassifier',
                'HistGradientBoostingClassifier',
                'XGBClassifier',
                'LGBMClassifier',
                'CatBoostClassifier',
                'MLPClassifier',
                )
models

(LogisticRegression(),
 RandomForestClassifier(),
 GradientBoostingClassifier(),
 HistGradientBoostingClassifier(),
 XGBClassifier(base_score=None, booster=None, callbacks=None,
               colsample_bylevel=None, colsample_bynode=None,
               colsample_bytree=None, early_stopping_rounds=None,
               enable_categorical=False, eval_metric=None, gamma=None,
               gpu_id=None, grow_policy=None, importance_type=None,
               interaction_constraints=None, learning_rate=None, max_bin=None,
               max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
               max_leaves=None, min_child_weight=None, missing=nan,
               monotone_constraints=None, n_estimators=100, n_jobs=None,
               num_parallel_tree=None, predictor=None, random_state=None,
               reg_alpha=None, reg_lambda=None, ...),
 LGBMClassifier(),
 MLPClassifier(max_iter=1500))

In [41]:
def make_cross_validation(estimator, X_train, y_train, cv=5):
    """Perform cross validation and teturn metrics:
    accuracy, balanced_accuracy, f1, precision, recall, roc_auc"""
    scorings = ('accuracy', 'balanced_accuracy', 'f1', 'precision', 'recall', 'roc_auc')
    scores = cross_validate(estimator, X_train, y_train, cv=cv, n_jobs=-1, scoring=scorings)
    final_metrics = dict()
    for key, item in sorted(scores.items()):
        final_metrics[key] = item.mean().round(4)
    return final_metrics

In [42]:
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=5, random_state=12)
data_cross_val = []
for mdl in models:
    report_dict = make_cross_validation(mdl, X_train, y_train, cv=cv)
    data_cross_val.append(report_dict)
df_report = pd.DataFrame(data_cross_val, index=models_tuple)
df_report

,fit_time,score_time,test_accuracy,test_balanced_accuracy,test_f1,test_precision,test_recall,test_roc_auc
LogisticRegression,0.0040,0.0054,0.8325,0.8323,0.8362,0.8291,0.8449,0.9049
RandomForestClassifier,0.2058,0.0311,0.9018,0.9017,0.9035,0.8995,0.9081,0.9671
GradientBoostingClassifier,0.2442,0.0062,0.8995,0.8994,0.9016,0.8958,0.9081,0.9584
HistGradientBoostingClassifier,0.3038,0.0110,0.9278,0.9278,0.9285,0.9309,0.9269,0.9721
XGBClassifier,0.1211,0.0084,0.9225,0.9224,0.9240,0.9198,0.9289,0.9711
LGBMClassifier,0.1442,0.0132,0.9238,0.9238,0.9247,0.9264,0.9235,0.9711
CatBoostClassifier,1.6041,0.0090,0.9302,0.9301,0.9317,0.9247,0.9393,0.9745
MLPClassifier,3.3061,0.0056,0.9342,0.9342,0.9354,0.9317,0.9398,0.9762


In [43]:
def test_evaluation(estimator, X_train, y_train, X_test, y_test) -> dict:
    """Calculates the main model metrics - accuracy, balanced-Accuracy,
    recall, precision, f1_score on the test set and returns them as a map object."""
    estimator.fit(X_train, y_train)
    y_pred = estimator.predict(X_test)

    accuracy = metrics.accuracy_score(y_test, y_pred).round(4)
    balanced_accuracy = metrics.balanced_accuracy_score(y_test, y_pred).round(4)
    precision = metrics.precision_score(y_test, y_pred).round(4)
    f_1 = metrics.f1_score(y_test, y_pred).round(4)
    recall = metrics.recall_score(y_test, y_pred).round(4)
    roc_auc = metrics.roc_auc_score(y_test, y_pred).round(4)

    final_metrics = {'accuracy': accuracy,
                     'balanced_accuracy': balanced_accuracy,
                     'f1': f_1,
                     'precision': precision,
                     'recall': recall,
                     'roc_auc': roc_auc,
                    }
    # Use dict comprehension for round dict values
    final_metrics = {key: round(val, 4) for key, val in final_metrics.items()}
    return final_metrics

In [44]:
# Evaluate the models on the test set
data_test = []
for mdl in models:
    report_dict_test = test_evaluation(mdl, X_train, y_train, X_test, y_test)
    data_test.append(report_dict_test)
print('Evaluate the models on the test set: ')
df_report_test = pd.DataFrame(data_test, index=models_tuple)
df_report_test

Evaluate the models on the test set: 


,accuracy,balanced_accuracy,f1,precision,recall,roc_auc
LogisticRegression,0.845,0.8438,0.8324,0.8851,0.7857,0.8438
RandomForestClassifier,0.910,0.9096,0.9062,0.9255,0.8878,0.9096
GradientBoostingClassifier,0.885,0.8847,0.8808,0.8947,0.8673,0.8847
HistGradientBoostingClassifier,0.940,0.9400,0.9388,0.9388,0.9388,0.9400
XGBClassifier,0.920,0.9196,0.9167,0.9362,0.8980,0.9196
LGBMClassifier,0.930,0.9300,0.9286,0.9286,0.9286,0.9300
CatBoostClassifier,0.950,0.9498,0.9485,0.9583,0.9388,0.9498
MLPClassifier,0.945,0.9447,0.9430,0.9579,0.9286,0.9447
